In [14]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from sklearn import tree
import os
import xlrd
import pandas as pd
import seaborn
import csv
from csv import reader
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from matplotlib.lines import Line2D
from scipy.stats import pearsonr
RF_SEED = 30

In [2]:
# Read the csv file into a pandas DataFrame

Property = pd.read_csv('Resources/Property.csv')
Property.head()

C:\Users\coset\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Parcel_ID,Parcel_Acreage,Property_Street_Address,Property_City,Property_State,Property_Zip,Parcel_Vacancy,Conveyance_Date,Property_Sales_Price,Seller_City,...,Buyer_ZIP,Primary_Residence,Assessed_Value_Land,Assessed_Value_Improvement,Total_Assessed_Value,Property_Class_Code,Class_Code_General,Class_Code_Translated,County_ID,County_Name
0,4574368,0.169,3909 MARIAH DRIVE ...,NEW ALBANY,IN,41750,N,6/2/2020,179900.0,NEW ALBANY,...,47150,Y,43800.0,138500.0,182300.0,510.0,500,Residential,22,Floyd
1,4515880,0.134,4110 E 10TH STREET,INDIAAPOLIS,IN,42601,N,10/19/2020,48000.0,Petaluma,...,46074,N,5000.0,88500.0,93500.0,510.0,500,Residential,49,Marion
2,4454504,0.172,5404 OAK HARBOR CT,INDIANAPOLIS,IN,42637,N,6/18/2020,195000.0,Indianapolis,...,46237,Y,30000.0,158000.0,188000.0,520.0,500,Residential,49,Marion
3,4421719,NaN,6516 EMERALD HILL COURT #310,INDIANAPOLIS,IN,42637,N,5/1/2020,120000.0,Franklin,...,42637,Y,10400.0,77900.0,88300.0,550.0,500,Residential,49,Marion
4,4531964,2.155,6335 Intech Commons Drive,Indianapolis,IN,42678,N,12/16/2020,1785000.0,MIAMI BEACH,...,75093,N,845000.0,957900.0,1802900.0,425.0,400,Commercial,49,Marion


In [3]:
len(Property)

184189

In [4]:
#Drop NA's from DF
pd.DataFrame(Property).dropna()
Property.head()
Property_df = Property[~Property.isin([np.nan, np.inf, -np.inf]).any(1)]
len(Property_df)

180360

In [5]:
#Select 5 Columns of DF
#Property_df=Property_df[['Property_Sales_Price','Total_Assessed_Value','Parcel_Acreage','Parcel_Vacancy','Class_Code_Translated']]
Property_df=Property_df[['Property_Sales_Price','Class_Code_Translated','Total_Assessed_Value','Parcel_Acreage','Parcel_Vacancy', 'County_Name']]

In [6]:
num_col = ['Property_Sales_Price','Total_Assessed_Value', 'Parcel_Acreage']

In [7]:

# outliers
print(Property_df[(np.abs(stats.zscore(Property_df[num_col])) > 3)].shape)
# good data
print(Property_df[(np.abs(stats.zscore(Property_df[num_col])) < 3).all(axis=1)].shape)

(646, 6)
(179715, 6)


In [8]:
#Git Rid of Outliers
Property_df = Property_df[(np.abs(stats.zscore(Property_df[num_col])) < 3).all(axis=1)].reset_index()
Property_df.head()


,index,Property_Sales_Price,Class_Code_Translated,Total_Assessed_Value,Parcel_Acreage,Parcel_Vacancy,County_Name
0,0,179900.0,Residential,182300.0,0.169,N,Floyd
1,1,48000.0,Residential,93500.0,0.134,N,Marion
2,2,195000.0,Residential,188000.0,0.172,N,Marion
3,4,1785000.0,Commercial,1802900.0,2.155,N,Marion
4,5,257575.0,Residential,210900.0,0.201,N,Marion


In [9]:
#One Hot encoding for non Numerical 
Property_df = pd.get_dummies(data=Property_df, columns=['Parcel_Vacancy','Class_Code_Translated','County_Name'])
#new_df.head()


In [10]:
#Git Rid of Index
Property_df.drop(['index'],axis=1,inplace=True)



In [63]:
Property_df = Property_df.loc[Property_df['Total_Assessed_Value']>0,:]
df = Property_df.loc[Property_df['Parcel_Acreage']>0,:]
#len(new_df)


In [64]:

#Seperate X and Y
y_data = Property_df['Property_Sales_Price']

x_data = Property_df.loc[:, Property_df.columns != 'Property_Sales_Price']

In [65]:
    data = df.loc[:, df.columns !='Property_Sales_Price'] 
    print( df['Property_Sales_Price'], data, data.columns )


0          179900.0
1           48000.0
2          195000.0
3         1785000.0
4          257575.0
            ...    
179710      90000.0
179711      78000.0
179712      11250.0
179713     200000.0
179714     184000.0
Name: Property_Sales_Price, Length: 171928, dtype: float64         Total_Assessed_Value  Parcel_Acreage  Parcel_Vacancy_N  \
0                   182300.0           0.169                 1   
1                    93500.0           0.134                 1   
2                   188000.0           0.172                 1   
3                  1802900.0           2.155                 1   
4                   210900.0           0.201                 1   
...                      ...             ...               ...   
179710               64500.0        4000.000                 1   
179711               46700.0        2000.000                 1   
179712               67200.0        2700.000                 1   
179713                9200.0        1564.000                 

In [66]:
def split_data_train_model(labels, data):
    # 20% examples in test data
    train, test, train_labels, test_labels = train_test_split(data,
                                                              labels,
                                                              test_size=0.2,
                                                              random_state=RF_SEED)

# training data fit
    regressor = RandomForestRegressor(n_estimators=100, random_state=RF_SEED)
    regressor.fit(train, train_labels)
 
    return test, test_labels, regressor


In [68]:
#y_data, x_data, feature_names = load_input('Resources/New.csv')
#y_data, x_data, feature_names = load_input(data)

y_data = y_data.to_numpy()
x_data = x_data.to_numpy()
print(feature_names)

Index(['Unnamed: 0', 'Total_Assessed_Value', 'Parcel_Acreage',
       'Parcel_Vacancy_N', 'Parcel_Vacancy_Y',
       'Class_Code_Translated_Agricultural',
       'Class_Code_Translated_Commercial', 'Class_Code_Translated_Exempt',
       'Class_Code_Translated_Industrial',
       'Class_Code_Translated_Locally Assessed',
       ...
       'County_Name_Vermillion', 'County_Name_Vigo', 'County_Name_Wabash',
       'County_Name_Warren', 'County_Name_Warrick', 'County_Name_Washington',
       'County_Name_Wayne', 'County_Name_Wells', 'County_Name_White',
       'County_Name_Whitley'],
      dtype='object', length=102)


In [69]:
x_test, x_test_labels, regressor = split_data_train_model(y_data, x_data)
 
predictions = regressor.predict(x_test)

In [72]:
# find the correlation between real answer and prediction
correlation = round(pearsonr(predictions, x_test_labels)[0], 5)
 

title_name = "Random Forest Regression - Property Price vs - correlation ({})".format(correlation)
x_axis_label = "Real House Price"
y_axis_label = "Predicted House Price"
 
# plot data
#simple_scatter_plot(x_test_labels, predictions, title_name, x_axis_label, y_axis_label)

print(correlation)

0.78572


In [70]:
features_importance = regressor.feature_importances_
 
print("Feature ranking:")
for i, data_class in enumerate(feature_names):
    print("{}. {} ({})".format(i + 1, data_class, features_importance[i]))


Feature ranking:
1. Unnamed: 0 (0.6521533981570894)
2. Total_Assessed_Value (0.14893998816026427)
3. Parcel_Acreage (0.014497597181210837)
4. Parcel_Vacancy_N (0.015453636332016615)
5. Parcel_Vacancy_Y (0.0022807846164643306)
6. Class_Code_Translated_Agricultural (0.01470304185589352)
7. Class_Code_Translated_Commercial (0.008789905207844475)
8. Class_Code_Translated_Exempt (0.011038698612739707)
9. Class_Code_Translated_Industrial (0.00015720483694827323)
10. Class_Code_Translated_Locally Assessed (2.0156212298305363e-07)
11. Class_Code_Translated_Mineral (0.013136159808486552)
12. Class_Code_Translated_Residential (0.00468056405396233)
13. County_Name_Allen (0.00039565650279387477)
14. County_Name_Bartholomew (1.9679730575929725e-05)
15. County_Name_Benton (2.5847581228760385e-05)
16. County_Name_Blackford (0.010739804725887056)
17. County_Name_Boone (0.0002427387974811436)
18. County_Name_Brown (3.9693037218028234e-05)
19. County_Name_Carroll (0.0002111686291397124)
20. County_Name_

IndexError: index 101 is out of bounds for axis 0 with size 101